In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from requests_html import HTMLSession, AsyncHTMLSession

In [3]:
rejected = [
    'wa.link',
    'mailto:',
    't.me',
    'tally.so',
    'discord.gg',
    'arxiv.org',
    'javascript:'
]

In [17]:
cookies = {
    'dtCookie': 'v_4_srv_1_sn_06257CAD9E2A461E834ADA3CDCB723E1_perc_100000_ol_0_mul_1_app-3Aea7c4b59f27d43eb_1',
    'rxVisitor': '1688269222666VOQ7GIVMFSIK16CNTLFFJAFQQH93MOET',
    'dtSa': '-',
    'rxvt': '1688271023019|1688269222667',
    'dtPC': '1$469222663_853h-vLVFAKFUAHHAVPAMGWCUKTCAASAGMDRWA-0e0',
    'aws-waf-token': '65b45488-7434-482d-b039-724f44df240c:BgoAmOsYx00LAAAA:pmsRFk8ucGyfyI4PYIAuqhl2uoRfpTxRAf1NtVpUTh+qe+pAn/yuw3gK/irq7Mqz/UfVImsuFi7eQnddhTBm534OqvvZq5PXxVodYfs1lujuT7x42HfGWW4nlqFgwo5XChQANnEq2VM5GahtaMsK7s+IhbkJYKy8W1VGQkhHncFV8zS+JYiW11RsJYALglkx0jQtL+avpQUSzw8D8EZpMU0fY2RFMtFQh2O+iINX+Bl8dZWw5bql',
    'COOKIE_SUPPORT': 'true',
    'GUEST_LANGUAGE_ID': 'en_US',
    'JSESSIONID': 'DgFFXr9Yy14ze9PHlrPhkw5Apr4YcciAwBObRMUK.liferay1',
    'AWSALBAPP-1': '_remove_',
    'AWSALBAPP-2': '_remove_',
    'AWSALBAPP-3': '_remove_',
    '_ga': 'GA1.3.1048913753.1688269229',
    '_gid': 'GA1.3.1498947204.1688269229',
    '_ga_QCXBYX5C31': 'GS1.3.1688269229.1.0.1688269229.60.0.0',
    'LFR_SESSION_STATE_20106': '1688269229239',
    'dtLatC': '1',
    'AWSALBTG': 'YjUis/PRd+tdQlqhRji/8Pyx0dsGsTXY5RKIWoCeTy1MD8WNUL1qhLaNBkRWRiHPmXQUeXHf2od9iEgmjO4j7A5m0KGY3BnPU7ODo5EoIWGgGnBz5Nbpm9nNTvXTIy2BydT531JivZWYmnY1uK5kEzkrHzM1HXtW3x3dJQo0mLtRfyZMg68=',
    'AWSALBAPP-0': 'AAAAAAAAAAACbQ8Cgoa9TGFqd2qTPdKf1vSXcK8bFDcmQfR6PcqlMK4rFSJ6c9l8sQbnEjD6+Bt73b/TJIQBcDLxuufaIGfhEiLE0z8yn9VOsn/U/IQ0a86wexJyzslyEni2GiqOJ0xb5w==',
}

headers = {
    'authority': 'www.bnm.gov.my',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': 'dtCookie=v_4_srv_1_sn_06257CAD9E2A461E834ADA3CDCB723E1_perc_100000_ol_0_mul_1_app-3Aea7c4b59f27d43eb_1; rxVisitor=1688269222666VOQ7GIVMFSIK16CNTLFFJAFQQH93MOET; dtSa=-; rxvt=1688271023019|1688269222667; dtPC=1$469222663_853h-vLVFAKFUAHHAVPAMGWCUKTCAASAGMDRWA-0e0; aws-waf-token=65b45488-7434-482d-b039-724f44df240c:BgoAmOsYx00LAAAA:pmsRFk8ucGyfyI4PYIAuqhl2uoRfpTxRAf1NtVpUTh+qe+pAn/yuw3gK/irq7Mqz/UfVImsuFi7eQnddhTBm534OqvvZq5PXxVodYfs1lujuT7x42HfGWW4nlqFgwo5XChQANnEq2VM5GahtaMsK7s+IhbkJYKy8W1VGQkhHncFV8zS+JYiW11RsJYALglkx0jQtL+avpQUSzw8D8EZpMU0fY2RFMtFQh2O+iINX+Bl8dZWw5bql; COOKIE_SUPPORT=true; GUEST_LANGUAGE_ID=en_US; JSESSIONID=DgFFXr9Yy14ze9PHlrPhkw5Apr4YcciAwBObRMUK.liferay1; AWSALBAPP-1=_remove_; AWSALBAPP-2=_remove_; AWSALBAPP-3=_remove_; _ga=GA1.3.1048913753.1688269229; _gid=GA1.3.1498947204.1688269229; _ga_QCXBYX5C31=GS1.3.1688269229.1.0.1688269229.60.0.0; LFR_SESSION_STATE_20106=1688269229239; dtLatC=1; AWSALBTG=YjUis/PRd+tdQlqhRji/8Pyx0dsGsTXY5RKIWoCeTy1MD8WNUL1qhLaNBkRWRiHPmXQUeXHf2od9iEgmjO4j7A5m0KGY3BnPU7ODo5EoIWGgGnBz5Nbpm9nNTvXTIy2BydT531JivZWYmnY1uK5kEzkrHzM1HXtW3x3dJQo0mLtRfyZMg68=; AWSALBAPP-0=AAAAAAAAAAACbQ8Cgoa9TGFqd2qTPdKf1vSXcK8bFDcmQfR6PcqlMK4rFSJ6c9l8sQbnEjD6+Bt73b/TJIQBcDLxuufaIGfhEiLE0z8yn9VOsn/U/IQ0a86wexJyzslyEni2GiqOJ0xb5w==',
    'pragma': 'no-cache',
    'referer': 'https://www.bnm.gov.my/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

In [18]:
async def get_html(url):
    session = AsyncHTMLSession(verify = False)
    result = None
    try:
        
        r = await session.get(url, verify = False, timeout = 10, cookies = cookies, headers = headers)
        await r.html.arender()
        
        result = r.html.html
        print(result)
    except Exception as e:
        print(e)
        pass
    
    await session.close()
    return result


def get_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    for link in soup.find_all('a'):
        links.append(link.get('href'))
    return links

In [19]:
import asyncio

visited_pages = set(['https://www.bnm.gov.my/'])

async def crawl(base_url):
    if base_url.endswith('.pdf'):
        return
    
    html = await get_html(base_url)
    
    if html is None:
        return

    visited_pages.add(base_url)
    links = get_links(html)
    for link in links:
        try:
            if any([r in link for r in rejected]):
                continue

            if not link.startswith('http'):
                link = urljoin(base_url, link)

            link = link.split('#')[0]
            link = link.split('?')[0]

            if link not in visited_pages:
                visited_pages.add(link)
        except Exception as e:
            pass

In [ ]:
from tqdm import tqdm

batch_size = 10

for no in range(3):
    l = list(visited_pages)
    print(no, len(l))
    for i in tqdm(range(0, len(l), batch_size)):
        b = l[i: i + batch_size]
        await asyncio.gather(*[crawl(url) for url in b])

0 1


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.42s/it]


<!DOCTYPE html><html lang="en"><head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Human Verification</title>
    <style>
        body {
            font-family: "Arial";
        }
    </style>
    <script type="text/javascript">
    window.awsWafCookieDomainList = [];
    </script>
    <script src="https://81264fbfae7c.fd32e112.ap-southeast-1.token.awswaf.com/81264fbfae7c/a81ddaf9ff40/1caaa84ffad0/challenge.js"></script>
</head>
<body>
    <div id="challenge-container"></div>
    <script type="text/javascript">
        AwsWafIntegration.checkForceRefresh().then((forceRefresh) => {
            if (forceRefresh) {
                AwsWafIntegration.forceRefreshToken().then(() => {
                    window.location.reload(true);
                });
            } else {
                AwsWafIntegration.getToken().then(() => {
                    window.location.reload(true);
                });
            }
        });
 

  0%|                                                     | 0/1 [00:00<?, ?it/s]

In [ ]:
import json

with open('url-moh.gov.my.json', 'w') as fopen:
    json.dump(list(visited_pages), fopen)